In [31]:
# I want to explore some methods of scraping with beatiful soup and selenium
# This is not the fastest, or the best implementation. This same process can be done with only one of those two tools
# It can be improved with xpath instead of searching inside the objects for example, but I wanted something different :) 

# The Code is a 
import requests
from bs4 import BeautifulSoup

# Most websites refuse GET requests from python, so we change the header to pretend we're a browser.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
# get webpage
page = requests.get("https://ca.ign.com/search?q=Super%20Mario%20Kart&page=0&count=10&", headers = headers)
# page = requests.get("https://ca.ign.com/search?q=red%20dead%20redemption%202", headers = headers)
# print status. 200 is successful, without the headers we would get 403 forbidden 
page

<Response [200]>

In [32]:
# get content
content = page.content
# show first 2000 letters of the content
str(page.content)[:2000:]

'b\'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\\n<html xmlns="https://www.w3.org/1999/xhtml" xml:lang="en" lang="en"\\n      xmlns:og="https://ogp.me/ns#"\\n      xmlns:fb="https://www.facebook.com/2008/fbml">\\n<head>\\n        <title>Search Results for &quot;Super Mario Kart&quot; - IGN</title>\\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\\n<meta name="description" content="Product search results for &amp;quot;Super Mario Kart&amp;quot; on IGN" />\\n<meta name="robots" content="noodp, noydir" />\\n<meta name="copyright" content="IGN Entertainment, Inc." />\\n<link rel="canonical" href="https://ca.ign.com/search" />\\n<link rel="alternate" hreflang="en-au" href="https://au.ign.com/search" />\\n<link rel="alternate" hreflang="en-ca" href="https://ca.ign.com/search" />\\n<link rel="alternate" hreflang="en-ie" href="https://ie.ign.com/search" />\\n<link rel="alternate" hreflang="en-

In [36]:
# Use BeautifulSoup to parse the HTML and make the content more readable
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.div.prettify()) # displaying a <div> tag
# print(soup.prettify()) # display entire webpage

<div class="header-container spine hidden-opacity ign-transition-opacity" id="header-container">
 <!-- Global Spine -->
 <div id="header-spine">
  <div class="locale-selector-container" id="locale-selector-container">
   <div class="locale-dropdown-bg ign-modal hidden">
    <div class="button-close">
     ×
    </div>
    <!-- multiplication symbol (×) -->
    <div class="locale-dropdown-container">
     <ul class="locale-dropdown">
     </ul>
    </div>
   </div>
  </div>
  <div class="locale-shortcode" data-hover-text="edition">
   ca
  </div>
  <a href="/" id="ign-logo" title="IGN">
   <span class="hidden">
    IGN
   </span>
   <svg class="ign-icon icon-logo" viewbox="0 0 1482.1 464.2" xmlns="http://www.w3.org/2000/svg">
    <g fill="currentColor">
     <path d="M791 271.2v-79.4l213 .1c21.6 0 39.3 17.5 39.3 39.1v94.4c0 48.6-38.8 88.8-88.2 88.8l-136.4.1c-101.1 0-182.8-81.6-182.8-182.4 0-100.7 82.3-182.4 183-182.4H1030v83.8H818.8c-54.4 0-99.1 44.1-99.1 98.6 0 54.4 44 98.5 99 98.5h140

In [4]:
# create a list with the web content separated into childs
listOne = list(soup.children)

# print type of items in the list
[type(item) for item in listOne]

[bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Comment]

In [5]:
# Looking which item contains the Review, apparently it's the third
# print(listOne[2])

# check the third item and assign it to a variable
soup = listOne[2]
listTwo = list(soup.children)

# print type of items in the list
[type(item) for item in listTwo]

[bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [6]:
# Repeating... look for review tag, get item to a list, list children, repeat
soup = listTwo[3]
myList = list(soup.children)

[type(item) for item in myList]


[bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [7]:
# Realize this is boring and should be automated...
# Use fnmatch to get the index of the item containing 'review-score'
import fnmatch

# for each item in the list, convert the item to a string, search the term and print the index
for idx, item in enumerate(myList):
    tagData = str(myList[idx])
    if fnmatch.fnmatch(tagData, '*review-score*'):
        print(idx)
        
# this is better but it's still taking too long

6


In [8]:
# define function to get the index just like before and return it
def getIndex(myList, search):
    for idx, item in enumerate(myList):
        tagData = str(myList[idx])
        if fnmatch.fnmatch(tagData, search):
            return idx
        
# define search term
search = '*review-score*'

# repeat the previous processes until our soup have only one value
while len(myList) > 1:
    soup = myList[getIndex(myList, search)]
    myList = list(soup.children) 

In [9]:
# get the number
text = str(myList)
print(text)

text = text[4:28:]
text.strip()

['\n            9.5          ']


'9.5'

In [10]:
# I don't want to keep searching for the games and copy-pasting urls, so I tried selenium for this process

import time
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

url = "https://ca.ign.com/search?q=Super%20Mario%20Kart&page=0&count=10&"

# Chromium driver -- http://chromedriver.chromium.org/
# from selenium.webdriver.common.keys import Keys
# driver = webdriver.Chrome(executable_path="chromedriver.exe")

# Gecko driver marionetting Mozilla Firefox -- https://github.com/mozilla/geckodriver/releases
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True
driver = webdriver.Firefox(capabilities=cap, executable_path="geckodriver.exe")

# Webpage takes too long to open, so we just load it for 8 seconds and stop it, this should be enough
# Define time and timeout
t = time.time()
driver.set_page_load_timeout(8)

# try loading the page, catch the timeout exception and stop loading
try:
    driver.get(url)
except TimeoutException:
    driver.execute_script("window.stop();")
print('Time consuming:', time.time() - t)

# get the search block, clear, input new search and hit Enter
elem = driver.find_element_by_id("query-input")
elem.clear()
elem.send_keys("Red Dead Redemption")
elem.send_keys(Keys.RETURN)
#search_button.click()

# get new url
print(driver.current_url)
# quit
driver.quit()

Time consuming: 8.151108741760254
https://ca.ign.com/search?q=Red%20Dead%20Redemption&page=0&count=10&


In [11]:
# Now let's try to pack up the knowledge and build the solution

In [30]:
import time
import fnmatch
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

t1 = time.time()

# SELENIUM
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True
driver = webdriver.Firefox(capabilities=cap, executable_path="geckodriver.exe")

# load .csv to pandas dataframe
dataFrame = pd.read_csv('Video_Games_Sales_2016.csv')
gameTitles = dataFrame['Name']
urlList = []

# Search a term and return the resulting url
def getUrl(search):
    # get the search block, clear, input new search and hit Enter
    elem = driver.find_element_by_id("query-input")
    elem.clear()
    elem.send_keys(search)
    elem.send_keys(Keys.RETURN)
    #time.sleep(5)
    # return search result url
    return driver.current_url

# Go to page, try loading it for 8 seconds and stop
def goTo(url):
    driver.set_page_load_timeout(8)
    try:
        driver.get(url)
    except TimeoutException:
        driver.execute_script("window.stop();")

# go to a webpage to start
goTo("https://ca.ign.com/search?q=Super%20Mario%20Kart&page=0&count=10&")

# search the games from the .csv and save the result url to a list
for item in gameTitles:
    urlList.append(str(getUrl(item)))
    
# quit
driver.quit()

# measure checkpoint 1
t2 = time.time()
print('time to get urls: ',t2 - t1)

#BEAUTIFUL SOUP

# get the content and return a list
def getContent(url):
    # Most websites refuse GET requests from python, so we change the header to pretend we're a browser.
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return list(soup.children)

# Search a list and return the first matching index    
def getIndex(myList, search):
    for idx, item in enumerate(myList):
        tagData = str(myList[idx])
        if fnmatch.fnmatch(tagData, search):
            return idx

# refine by seeking a term until your list reach the defined limit of items
def lookup(myList, desiredSize):
    while len(myList) > desiredSize:
        soup = myList[getIndex(myList, '*review-score*')]
        myList = list(soup.children)
    return myList

# look for attribute href in the anchors and append the found link to the webpages list
def getAttr(anchors):
    attr = {'webpage':'','game':''}
    for a in anchors:
        if a.has_attr('href'):
            attr['webpage']=a['href']
            attr['game']=str(a['href']).split("/")[4]
            break
    return attr


# Define the lists where the results will be saved
scores = []
webpages = []
games = []

# Check every url of the list created with selenium
for url in urlList:
    # get content and refine list to 3 items
    myList = getContent(url)
    myList = lookup(myList, 3)
    
    # get the item containing the href and list all the anchors
    soup = myList[getIndex(myList, '*href*')]
    anchors = soup.findAll('a')
    # get the link and the game name from the anchor and append them to their lists
    attributes = getAttr(anchors)
    webpages.append(attributes['webpage'])
    games.append(attributes['game'])
    
    # get the review and format it properlly
    myList = lookup(myList, 1)    
    text = str(myList)[4:27:]
    scores.append(text.strip())

t3 = time.time()
print('time to get scores: ',t3-t2)
print('total time: ',t3-t1)
# add all lists to dataframe and display headers
dataFrame['IGN_title'] = games
dataFrame['IGN_source'] = webpages
dataFrame['IGN_score'] = scores
dataFrame.head()

time to get urls:  16.810179948806763
time to get scores:  4.959393501281738
total time:  21.7695734500885


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,IGN_title,IGN_source,IGN_score
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E,wii-sports,https://ca.ign.com/games/wii-sports/wii-826987,7.5
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,wii-sports,https://ca.ign.com/games/wii-sports/wii-826987,7.5
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,mario-kart-wii,https://ca.ign.com/games/mario-kart-wii/wii-94...,8.5
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E,wii-sports-resort,https://ca.ign.com/games/wii-sports-resort/wii...,7.7
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,pokemon-blue-version,https://ca.ign.com/games/pokemon-blue-version/...,10
